## Librerías

In [49]:
%run "../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: shap in c:\users\admin\.conda\envs\datascience\lib\site-packages (0.46.0)

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [50]:
%run "../funciones.ipynb"

In [51]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [52]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l

data = pd.read_parquet(dataset_clase_ternaria_l)

## Valores null por variable

In [53]:
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    breakpoints = sorted(set(
    [edge.left for edge in bin_edges if isinstance(edge, pd.Interval)] + 
    [edge.right for edge in bin_edges if isinstance(edge, pd.Interval)]))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [54]:
# Ingresar mes train y mes score
train_data = data[data['foto_mes'] <= 202104]
score_data = data[data['foto_mes'] == 202106]

train_null_percentage = train_data.isnull().mean() * 100
score_null_percentage = score_data.isnull().mean() * 100

comparison_df = pd.DataFrame({'Train Null Percentage': train_null_percentage, 'Score Null Percentage': score_null_percentage})
comparison_df['diff'] = (comparison_df['Score Null Percentage'] - comparison_df['Train Null Percentage']).abs()

comparison_df_sorted = comparison_df.sort_values('diff', ascending=False)
pd.set_option('display.max_rows', None)
comparison_df_sorted

,Train Null Percentage,Score Null Percentage,diff
tmobile_app,18.668335,0.000000,18.668335
cmobile_app_trx,18.668335,0.000000,18.668335
Master_mpagospesos,60.326929,58.459691,1.867238
Master_mconsumosdolares,60.326929,58.459691,1.867238
Master_cconsumos,60.326929,58.459691,1.867238
Master_mconsumototal,60.326929,58.459691,1.867238
Master_mpagosdolares,60.326929,58.459691,1.867238
Master_madelantodolares,60.326929,58.459691,1.867238
Master_madelantopesos,60.326929,58.459691,1.867238
Master_mconsumospesos,60.326929,58.459691,1.867238


## Análisis drifting con PSI

In [55]:
psi_results = []
for column in train_data.columns:
  if column not in ['foto_mes', 'clase_ternaria']:
    train_variable = train_data[column]
    score_variable = score_data[column]
    psi_value = psi(train_variable, score_variable)
    psi_results.append({'feature': column, 'psi': psi_value})

psi_df = pd.DataFrame(psi_results)
psi_df = psi_df.sort_values('psi', ascending=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_22280\2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22280\2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22280\2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22280\2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\Admin\AppData\Local\Temp\ipykernel_22280\2831771324.py:4: RuntimeWarning: divide by zero encountered in log
  result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
C:\Users\Admin\AppDa

In [56]:
f = psi_df['psi'] > 0.2
lista_variables_con_drift = psi_df.loc[f, 'feature'].to_list()

In [57]:
lista_variables_con_drift

['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'mcajeros_propios_descuentos',
 'mpayroll2',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad']

In [58]:
variable_name = 'Master_Finiciomora'
expected = train_data[variable_name]
actual = score_data[variable_name]

expected_not_null = expected.dropna()
actual_not_null = actual.dropna()

bin_edges = pd.qcut(expected_not_null, q=10, duplicates='drop').unique()
bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
breakpoints = sorted(list(set(bin_edges2)))

print(f'Cortes en {variable_name}: {breakpoints}')
expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

print(f'Frecuencia Esperada: {expected_counts}')
print(f'Frecuencia Actual: {actual_counts}')


Cortes en Master_Finiciomora: [-1.001, 9.0, 15.0, 18.0, 19.0, 20.0, 21.0, 24.0, 46.0, 52.0, 207.0]
Frecuencia Esperada: [1851 1768 1347 1821 1275 2031 2676 2082 2016 1972]
Frecuencia Actual: [ 73   0   0   0   0   0 306  39 128 117]


## Analisis drifting con modelo

In [59]:
# 2- Para el caso de que el dataset sea con feature engineering incluido
if 'T_visa_normal' in data.columns:
    data['T_visa_normal'] = data['T_visa_normal'].astype(bool)
    data['T_master_normal'] = data['T_master_normal'].astype(bool)


In [60]:
# 3- parametros para calcular ganancia del modelo
ganancia_acierto = 273000
costo_estimulo = 7000


data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001


In [61]:
# 4 - binarización de la clase ternaria

data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)

In [63]:
# 5 Especificar mes de train y test

df_train = data[data['foto_mes']<=202104]
df_test = data[data['foto_mes']==202106]


In [64]:
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']

In [65]:
w_train = df_train.loc[X_train.index, 'clase_peso']


In [66]:
best_params = {'num_leaves': 40,
 'learning_rate': 0.022596031021514126,
 'min_data_in_leaf': 1692,
 'feature_fraction': 0.8060065372089812,
 'bagging_fraction': 0.8663914349386255}

In [67]:
train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)


params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': 0,
'learning_rate': 0,
'min_data_in_leaf': 0,
'feature_fraction': 0,
'bagging_fraction': 0,
'seed': semillas[0],
'verbose': 0
}
    #actualización con valores optuna
params.update(best_params)

if params['num_leaves'] == 0:
    print ('Error falta actualización de parámetros')


model = lgb.train(params,
                train_data,
                num_boost_round=299)



In [68]:
model_df = model.trees_to_dataframe()

In [28]:
model_df[model_df['split_feature'] == 'cpayroll_trx'].sort_values('threshold')

,tree_index,node_depth,node_index,left_child,right_child,parent_index,split_feature,split_gain,threshold,decision_type,missing_direction,missing_type,value,weight,count
12,0,6,0-S30,0-L23,0-L31,0-S22,cpayroll_trx,163.298004,1.000000e-35,<=,left,None,-1.799880,5183.050,41951
13017,164,3,164-S12,164-S13,164-L13,164-S1,cpayroll_trx,143.393997,1.000000e-35,<=,left,None,-0.000013,5085.280,208007
12958,164,3,164-S3,164-S10,164-L4,164-S2,cpayroll_trx,581.468018,1.000000e-35,<=,left,None,-0.006037,4965.610,518896
12776,161,3,161-S17,161-S20,161-L18,161-S1,cpayroll_trx,150.694000,1.000000e-35,<=,left,None,-0.000031,5100.990,208007
12721,161,3,161-S3,161-S12,161-L4,161-S2,cpayroll_trx,604.562988,1.000000e-35,<=,left,None,-0.006386,5073.260,518896
12696,160,5,160-S5,160-S7,160-S12,160-S3,cpayroll_trx,1315.939941,1.000000e-35,<=,left,None,-0.007736,15503.200,981946
12689,160,5,160-S20,160-S23,160-L21,160-S4,cpayroll_trx,127.492996,1.000000e-35,<=,left,None,-0.018714,8665.800,1149678
12670,160,5,160-S9,160-S10,160-L10,160-S4,cpayroll_trx,336.552002,1.000000e-35,<=,left,None,-0.004329,9057.240,376567
12662,160,4,160-S34,160-L7,160-L35,160-S11,cpayroll_trx,85.609001,1.000000e-35,<=,left,None,-0.009794,1842.980,159520
12614,159,5,159-S19,159-S21,159-L20,159-S4,cpayroll_trx,126.197998,1.000000e-35,<=,left,None,-0.018793,8827.010,1149678


## Analisis SHAPs

Calculamos la importancia de las variables usando SHAP


In [70]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)
shap_df = pd.DataFrame(shap_values, columns = X_train.columns)

shap_importancia = pd.DataFrame(np.abs(shap_values).mean(0), columns=['SHAP Importance'])
shap_importancia['Feature'] = X_train.columns
shap_importancia.sort_values('SHAP Importance', ascending=False, inplace=True)
shap_importancia



c:\Users\Admin\.conda\envs\datascience\Lib\site-packages\shap\explainers\_tree.py:448: UserWarning: LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray
  warnings.warn('LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray')


,SHAP Importance,Feature
1,2.669327,foto_mes
107,0.214889,ctrx_quarter
51,0.198960,cpayroll_trx
52,0.196661,mpayroll
145,0.138335,Visa_fultimo_cierre
123,0.120250,Master_fultimo_cierre
18,0.118233,mcaja_ahorro
28,0.104203,mtarjeta_visa_consumo
33,0.102554,mprestamos_personales
108,0.093147,tmobile_app


In [71]:
shap_importancia.columns

Index(['SHAP Importance', 'Feature'], dtype='object')

In [72]:
lista_shap_unimportant = shap_importancia.loc[shap_importancia['SHAP Importance']==0,'Feature'].to_list()

In [73]:
lista_para_sacar = []
lista_para_transformar = []

for x in lista_variables_con_drift:
    if x in lista_shap_unimportant:
        lista_para_sacar.append(x)
    if x not in lista_shap_unimportant:
        lista_para_transformar.append(x)

In [74]:
lista_para_transformar

['cmobile_app_trx',
 'mpayroll',
 'mprestamos_prendarios',
 'Master_fultimo_cierre',
 'Master_Finiciomora',
 'Visa_mlimitecompra',
 'Visa_madelantopesos',
 'Visa_fultimo_cierre',
 'Visa_Finiciomora',
 'internet',
 'mcomisiones_mantenimiento',
 'Master_Fvencimiento',
 'mrentabilidad_annual',
 'mcomisiones_otras',
 'mcomisiones',
 'mtransferencias_recibidas',
 'mrentabilidad']

In [75]:
lista_para_sacar

['mcajeros_propios_descuentos', 'mpayroll2']